In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib notebook
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
demo = pd.read_csv("/content/drive/MyDrive/bci/demographic.csv")
# demographic.csv columns have space as a first character.. remove it!
demo.columns = [col.replace(' ', '') for col in demo.columns]
erp_data = pd.read_csv("/content/drive/MyDrive/bci/ERPdata.csv")

In [4]:
# Electrodes
elec_right = ['FC4', 'C4', 'CP4']
elec_left = ['FC3', 'C3', 'CP3']
electrodes_lrp = elec_right + elec_left

# Substract baseline
BL = erp_data[(erp_data.time_ms >= -600) & (erp_data.time_ms <= -500)].groupby(['subject', 'condition']).mean()
RPamps = erp_data[(erp_data.time_ms >= -100) & (erp_data.time_ms <= 0)].groupby(['subject', 'condition']).mean()
RPamps = RPamps[electrodes_lrp] - BL[electrodes_lrp]

# Drop playback (won't need it here)
RPamps = RPamps.drop(index=2, level=1)
RPamps = RPamps.reset_index()

# Give proper names to group and condition values
RPamps_demo = RPamps.merge(demo, on='subject')
RPamps_demo.group.replace(to_replace={0: 'Control', 1: 'Schizo'}, inplace=True)
RPamps_demo.condition.replace(to_replace={1: 'Button tone', 3: 'Control press'}, inplace=True)

# Display for testing
display(RPamps_demo[:10])

# Boxplot
RPamps_demo.boxplot('FC4', by='group', figsize=(6, 4))

,subject,condition,FC4,C4,CP4,FC3,C3,CP3,group,gender,age,education
0,1,Button tone,-1.754095,-1.411906,-1.713631,-3.623855,-2.864079,-2.796495,Control,M,44,16.0
1,1,Control press,-1.927416,-1.311639,-2.030604,-2.376894,-3.119468,-2.909333,Control,M,44,16.0
2,2,Button tone,-0.676889,-0.232035,-0.514371,-1.231047,-1.141242,-0.607670,Control,M,39,17.0
3,2,Control press,0.391040,0.856605,0.579452,0.733641,0.824525,0.992221,Control,M,39,17.0
4,3,Button tone,-4.243524,-3.060327,-1.592617,-4.126250,-1.919484,-0.433224,Control,M,53,18.0
5,3,Control press,0.759739,3.158410,3.708572,1.348654,1.965423,3.224192,Control,M,53,18.0
6,4,Button tone,-1.434151,-1.251261,-0.758620,-3.406878,-3.206953,-2.289590,Control,M,52,15.0
7,4,Control press,-1.090816,-1.975662,-1.294551,-1.532150,-1.402029,-1.079045,Control,M,52,15.0
8,5,Button tone,-0.823886,-1.089808,-0.746429,-1.380179,-1.307772,-0.840295,Control,M,41,16.0
9,5,Control press,-3.031956,-2.811327,-1.682536,-3.444261,-2.236882,-1.047005,Control,M,41,16.0


<IPython.core.display.Javascript object>

<Axes: title={'center': 'FC4'}, xlabel='group'>

In [5]:
#convert electrode columns to rows with labels:
RPamps_demo = pd.melt(RPamps_demo, id_vars=['subject', 'group', 'condition'], value_vars=['FC4', 'C4', 'CP4','FC3', 'C3', 'CP3'], var_name='electrode', value_name='RP')
RPamps_demo.head(10)

,subject,group,condition,electrode,RP
0,1,Control,Button tone,FC4,-1.754095
1,1,Control,Control press,FC4,-1.927416
2,2,Control,Button tone,FC4,-0.676889
3,2,Control,Control press,FC4,0.391040
4,3,Control,Button tone,FC4,-4.243524
5,3,Control,Control press,FC4,0.759739
6,4,Control,Button tone,FC4,-1.434151
7,4,Control,Control press,FC4,-1.090816
8,5,Control,Button tone,FC4,-0.823886
9,5,Control,Control press,FC4,-3.031956


In [6]:
#Add the hemisphere column
RPamps_demo['hemisphere'] = RPamps_demo['electrode']
RPamps_demo['hemisphere'] = RPamps_demo['hemisphere'].apply(lambda x: 'left' if '3' in x else 'right')

In [7]:
#Remove the numbers from the electrodes
RPamps_demo['electrode'] = RPamps_demo['electrode'].str[:-1]

In [8]:
RPamps_demo.head()


,subject,group,condition,electrode,RP,hemisphere
0,1,Control,Button tone,FC,-1.754095,right
1,1,Control,Control press,FC,-1.927416,right
2,2,Control,Button tone,FC,-0.676889,right
3,2,Control,Control press,FC,0.391040,right
4,3,Control,Button tone,FC,-4.243524,right


In [9]:
N1amps = erp_data[(erp_data.time_ms >= 80) & (erp_data.time_ms <= 100)].groupby(['subject', 'condition']).mean()
N1amps.loc[(slice(None),1), ['Fz', 'FCz', 'Cz']] = N1amps.loc[(slice(None),1), ['Fz', 'FCz', 'Cz']] - N1amps.loc[(slice(None),3), ['Fz', 'FCz', 'Cz']].values
N1amps = N1amps.drop(index=3, level=1)

In [10]:
N1amps=N1amps.reset_index()
N1amps = pd.melt(N1amps, id_vars=['subject', 'condition'], value_vars=['Fz', 'FCz', 'Cz'])
N1amps.condition.replace(to_replace={1: 'Button tone', 2: 'Playback'}, inplace=True)
N1amps.head()

,subject,condition,variable,value
0,1,Button tone,Fz,-4.937118
1,1,Playback,Fz,-8.308424
2,2,Button tone,Fz,-3.515848
3,2,Playback,Fz,-5.300647
4,3,Button tone,Fz,0.334000


In [11]:
N1amps = N1amps.merge(demo, on='subject')
N1amps.group.replace(to_replace={0: 'Control', 1: 'Schizo'}, inplace=True)
N1amps.head()

,subject,condition,variable,value,group,gender,age,education
0,1,Button tone,Fz,-4.937118,Control,M,44,16.0
1,1,Playback,Fz,-8.308424,Control,M,44,16.0
2,1,Button tone,FCz,-4.861151,Control,M,44,16.0
3,1,Playback,FCz,-8.990923,Control,M,44,16.0
4,1,Button tone,Cz,-4.567639,Control,M,44,16.0


In [12]:
###LRP amplitude
N1df = N1amps[(N1amps['condition'] == 'Button tone') & (N1amps['variable'] == 'FCz')].merge(N1amps[(N1amps['condition'] == 'Playback') & (N1amps['variable'] == 'FCz')], on='subject')


In [13]:
N1df['N1supp'] = N1df['value_x']-N1df['value_y']
N1df

,subject,condition_x,variable_x,value_x,group_x,gender_x,age_x,education_x,condition_y,variable_y,value_y,group_y,gender_y,age_y,education_y,N1supp
0,1,Button tone,FCz,-4.861151,Control,M,44,16.0,Playback,FCz,-8.990923,Control,M,44,16.0,4.129772
1,2,Button tone,FCz,-3.480376,Control,M,39,17.0,Playback,FCz,-5.608960,Control,M,39,17.0,2.128584
2,3,Button tone,FCz,0.434816,Control,M,53,18.0,Playback,FCz,-6.284744,Control,M,53,18.0,6.719560
3,4,Button tone,FCz,-3.267771,Control,M,52,15.0,Playback,FCz,-7.032685,Control,M,52,15.0,3.764914
4,5,Button tone,FCz,-4.599510,Control,M,41,16.0,Playback,FCz,-6.166350,Control,M,41,16.0,1.566840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,Button tone,FCz,-2.969091,Schizo,M,28,13.0,Playback,FCz,-2.949088,Schizo,M,28,13.0,-0.020003
77,78,Button tone,FCz,-5.768306,Schizo,F,32,16.0,Playback,FCz,-3.097072,Schizo,F,32,16.0,-2.671234
78,79,Button tone,FCz,1.942727,Schizo,M,37,16.0,Playback,FCz,-9.498868,Schizo,M,37,16.0,11.441594
79,80,Button tone,FCz,-1.097510,Schizo,M,33,13.0,Playback,FCz,-3.974132,Schizo,M,33,13.0,2.876622


In [14]:
LRPdf = RPamps_demo[(RPamps_demo['condition'] == 'Button tone') & (RPamps_demo['electrode'] == 'C') & (RPamps_demo['hemisphere'] == 'left')].merge(RPamps_demo[(RPamps_demo['condition'] == 'Button tone') & (RPamps_demo['electrode'] == 'C') & (RPamps_demo['hemisphere'] == 'right')], on='subject')
LRPdf['LRP'] = LRPdf['RP_x']-LRPdf['RP_y']
LRPdf

,subject,group_x,condition_x,electrode_x,RP_x,hemisphere_x,group_y,condition_y,electrode_y,RP_y,hemisphere_y,LRP
0,1,Control,Button tone,C,-2.864079,left,Control,Button tone,C,-1.411906,right,-1.452172
1,2,Control,Button tone,C,-1.141242,left,Control,Button tone,C,-0.232035,right,-0.909208
2,3,Control,Button tone,C,-1.919484,left,Control,Button tone,C,-3.060327,right,1.140843
3,4,Control,Button tone,C,-3.206953,left,Control,Button tone,C,-1.251261,right,-1.955692
4,5,Control,Button tone,C,-1.307772,left,Control,Button tone,C,-1.089808,right,-0.217964
...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,Schizo,Button tone,C,-1.590360,left,Schizo,Button tone,C,-0.832366,right,-0.757994
77,78,Schizo,Button tone,C,0.129652,left,Schizo,Button tone,C,-1.377571,right,1.507223
78,79,Schizo,Button tone,C,-2.684811,left,Schizo,Button tone,C,-2.290481,right,-0.394330
79,80,Schizo,Button tone,C,-0.494573,left,Schizo,Button tone,C,-1.396021,right,0.901448


In [15]:
N1LRPdf = N1df.merge(LRPdf, on='subject')
N1LRPdf = N1LRPdf[['subject', 'LRP', 'N1supp', 'group_x_x']]
N1LRPdf['group_x_x'] = N1LRPdf['group_x_x'].apply(lambda x: 0 if x=='Control' else 1)
N1LRPdf = N1LRPdf.rename(index=str, columns={'group_x_x': 'Group'})
N1LRPdf.head()

,subject,LRP,N1supp,Group
0,1,-1.452172,4.129772,0
1,2,-0.909208,2.128584,0
2,3,1.140843,6.719560,0
3,4,-1.955692,3.764914,0
4,5,-0.217964,1.566840,0


In [16]:
X = N1LRPdf[['LRP', 'N1supp']]
y = N1LRPdf['Group']

In [17]:
X_HC = X[y==0]
X_SZ = X[y==1]
y_HC = y[y==0]
y_SZ = y[y==1]

In [18]:
X_train_HC, X_test_HC, y_train_HC, y_test_HC = train_test_split(X_HC, y_HC, test_size=0.2, random_state=42)
X_train_SZ, X_test_SZ, y_train_SZ, y_test_SZ = train_test_split(X_SZ, y_SZ, test_size=0.2, random_state=42)

In [19]:
X_train_HC = X_train_HC.to_numpy()
X_train_SZ = X_train_SZ.to_numpy()
X_test_HC = X_test_HC.to_numpy()
X_test_SZ = X_test_SZ.to_numpy()

In [20]:
y_train_HC = y_train_HC.to_numpy()
y_train_SZ = y_train_SZ.to_numpy()
y_test_HC = y_test_HC.to_numpy()
y_test_SZ = y_test_SZ.to_numpy()

In [21]:
X_train = np.vstack((X_train_HC, X_train_SZ))
X_test = np.vstack((X_test_HC, X_test_SZ))

y_train = np.append(y_train_HC, y_train_SZ)
y_test = np.append(y_test_HC, y_test_SZ)

# shuffle
index_train = np.array(range(0, X_train.shape[0]))
np.random.shuffle(index_train)
index_test = np.array(range(0, X_test.shape[0]))
np.random.shuffle(index_test)

X_train = X_train[index_train]
X_test = X_test[index_test]
y_train = y_train[index_train]
y_test = y_test[index_test]

In [22]:
y_train

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0])

In [23]:
np.sum(y_test)/y_test.shape[0]

0.5882352941176471

In [24]:
clf = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
clf.fit(X_train,y_train)
print(clf.score(X_test, y_test))

0.5294117647058824


In [25]:
svm = LinearSVC(random_state=0)
svm.fit(X_train,y_train)
print(svm.score(X_test, y_test))

0.5294117647058824


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [26]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = np.arange(5,130,5)
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125]), 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [27]:
clf3=RandomForestClassifier(random_state=52)
rf_random = RandomizedSearchCV(estimator = clf3, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=52),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125])},
                   random_state=42, verbose=2)

In [28]:
best_random = rf_random.best_estimator_
print(rf_random.best_score_)
print(rf_random.best_params_)

0.6118326118326118
{'n_estimators': 20, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 70, 'bootstrap': True}


In [29]:
from sklearn.metrics import classification_report
report=classification_report(y_test, best_random.predict(X_test), output_dict=True)
df = pd.DataFrame(report).transpose()
df


,precision,recall,f1-score,support
0,0.333333,0.142857,0.200000,7.000000
1,0.571429,0.800000,0.666667,10.000000
accuracy,0.529412,0.529412,0.529412,0.529412
macro avg,0.452381,0.471429,0.433333,17.000000
weighted avg,0.473389,0.529412,0.474510,17.000000


In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc
import seaborn as sns
from sklearn.metrics import confusion_matrix

def roccurve(y_values, y_preds_proba):
    fpr, tpr, _ = roc_curve(y_values, y_preds_proba)
    xx = np.arange(101) / float(100)
    aur = auc(fpr,tpr)

    plt.style.use('fivethirtyeight')
    plt.xlim(0, 1.0)
    plt.ylim(0, 1.25)
    plt.plot([0.0, 0.0], [0.0, 1.0], color='green', linewidth=8)
    plt.plot([0.0, 1.0], [1.0, 1.0], color='green', label='Perfect Model', linewidth=4)
    plt.plot(xx,xx, label='Random Model')
    plt.plot(fpr,tpr, label='User Model')
    plt.title("AUC = "+str(round(aur, 3)))
    plt.xlabel('% false positives')
    plt.ylabel('% true positives')
    plt.legend()
    plt.show()

###Model with tuned parameters
y_pred_proba = best_random.predict_proba(X=X_test)
roccurve(y_values=y_test, y_preds_proba=y_pred_proba[:,1])

In [31]:
cm = confusion_matrix(y_test,best_random.predict(X_test))
sns.heatmap(cm,annot=True,fmt="d")

<Axes: title={'center': 'AUC = 0.543'}>

In [32]:
###Base model
y_pred_proba2 = clf.predict_proba(X=X_test)
roccurve(y_values=y_test, y_preds_proba=y_pred_proba2[:,1])

In [33]:
cm = confusion_matrix(y_test,clf.predict(X_test))
sns.heatmap(cm,annot=True,fmt="d")

<Axes: title={'center': 'AUC = 0.543'}>